In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os

import pixiedust

import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
from sklearn import metrics

from decagon.deep.optimizer import DecagonOptimizer
from decagon.deep.model import DecagonModel
from decagon.deep.minibatch import EdgeMinibatchIterator
from decagon.utility import rank_metrics, preprocessing

Pixiedust database opened successfully


/Users/ravanv/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Train on CPU (hide GPU) due to memory constraints
os.environ['CUDA_VISIBLE_DEVICES'] = ""

# Train on GPU
#os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

np.random.seed(0)

###########################################################
#
# Functions
#
###########################################################


def get_accuracy_scores(edges_pos, edges_neg, edge_type):
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    rec = sess.run(opt.predictions, feed_dict=feed_dict)

    def sigmoid(x):
        return 1. / (1 + np.exp(-x))

    # Predict on test set of edges
    preds = []
    actual = []
    predicted = []
    edge_ind = 0
    for u, v in edges_pos[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 1, 'Problem 1'

        actual.append(edge_ind)
        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_neg = []
    for u, v in edges_neg[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 0, 'Problem 0'

        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    apk_sc = rank_metrics.apk(actual, predicted, k=50)

    return roc_sc, aupr_sc, apk_sc


def construct_placeholders(edge_types):
    placeholders = {
        'batch': tf.placeholder(tf.int32, name='batch'),
        'batch_edge_type_idx': tf.placeholder(tf.int32, shape=(), name='batch_edge_type_idx'),
        'batch_row_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_row_edge_type'),
        'batch_col_edge_type': tf.placeholder(tf.int32, shape=(), name='batch_col_edge_type'),
        'degrees': tf.placeholder(tf.int32),
        'dropout': tf.placeholder_with_default(0., shape=()),
    }
    placeholders.update({
        'adj_mats_%d,%d,%d' % (i, j, k): tf.sparse_placeholder(tf.float32)
        for i, j in edge_types for k in range(edge_types[i,j])})
    placeholders.update({
        'feat_%d' % i: tf.sparse_placeholder(tf.float32)
        for i, _ in edge_types})
    return placeholders

In [3]:
###########################################################
#
# Load and preprocess data (This is a dummy toy example!)
#
###########################################################

####
# The following code uses artificially generated and very small networks.
# Expect less than excellent performance as these random networks do not have any interesting structure.
# The purpose of main.py is to show how to use the code!
#
# All preprocessed datasets used in the drug combination study are at: http://snap.stanford.edu/decagon:
# (1) Download datasets from http://snap.stanford.edu/decagon to your local machine.
# (2) Replace dummy toy datasets used here with the actual datasets you just downloaded.
# (3) Train & test the model.
####


In [4]:
#bio-decagon-ppi:Protein-protein interaction network
#bio-decagon-targets: Drug-target protein associations
#bio-decagon-targets-all: Drug-target protein associations culled from several curated databases
#bio-decagon-combo:Polypharmacy side effects in the form of (drug A, side effect type, drug B) triples
#bio-decagon-effectcategories: Side effect categories
#bio-decagon-mono:Side effects of individual drugs in the form of (drug A, side effect type) tuples

<h1>Information about the datasets from the paper:</h1>
<h4>The protein-prtoein network:</h4>
number of proteins = 19,085, number of physical interactions = 719,402
<h4>The drug-prtoein network:</h4>
number of proteins = 8,934, number of drugs = 519,022, number of interactions = 8,083,600
<h4>The drug-drug network(individuale):</h4>
number of  drugs = 1,556, number of side effects = 5,868, number of drug-side effect association = 286,399
<h4>The drug-drugn network(combination):</h4>
number of  drug combinations = 63,473, number of side effect types = 1,318, number of drug combination-side effect association = 4,651,131
<h2>Final Network:</h2>
<h3>Number of protiens = 19,085 (paper) ....... Number of protiens = 19,081(ppi data) </h3>
<h3>Number of drugs = 645 (paper).......... Number of drugs = 645 (polypharmacy side effect data (combo))</h3>
<h3>Number of protien-protien edges= 715,612(paper) ....... Number of protien-protien edges= 715,612 (ppi data)</h3>
<h3>Number of drug-drug edges= 4,651,131 (paper) ......... Number of drug-drug edges= 4,649,441 (polypharmacy side effect data (combo)) </h3>
<h3>Number of drug-protien edges= 18,596 (paper) ........ Number of drug-protien edges= 18,690 (Drug-target protein (targets))</h3>

In [5]:
#protein_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-ppi.csv", sep=',',header = 0)
#drug_target_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-targets.csv", sep=',',header = 0)
#drug_target_protein_all_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-targets-all.csv", sep=',',header = 0)
#polypharmacy_side_effect_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-combo.csv", sep=',',header = 0)
#side_effect_categories_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-effectcategories.csv", sep=',',header = 0)
#side_effect_individuale_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-mono.csv", sep=',',header = 0)


<h2>Reading 3 small datasets</h2>

In [6]:
import socket

if socket.gethostname()=='Peters-MacBook-Pro-2.local':
    dir ='/Users/peterrobinson/Documents/GIT/decagon/small_datasets/'
    protein_protein_data = pd.read_csv(dir +"small-ppi.csv", sep=',',header = 0)
    polypharmacy_side_effect_data = pd.read_csv(dir + "small-combo.csv", sep=',',header = 0)
    drug_target_protein_data = pd.read_csv(dir +"bio-decagon-targets.csv", sep=',',header  = 0)
else:
    protein_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/small-ppi.csv", sep=',',header = 0)
    polypharmacy_side_effect_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/small-combo.csv", sep=',',header = 0)
    drug_target_protein_data = pd.read_csv("/Users/ravanv/Desktop/Decagon/data/bio-decagon-targets.csv", sep=',',header = 0)

In [7]:
print("size of the protein-protein network:", protein_protein_data.shape)
print("size of the drug-target protein associations:", drug_target_protein_data.shape)
#print("size of the Drug-target protein associations culled from several curated databases :",drug_target_protein_all_data.shape)
print("size of the polypharmacy side effects:", polypharmacy_side_effect_data.shape)
#print("size of the Side effect categories:",side_effect_categories_data.shape)
#print("size of the Side effects of individual drugs:",side_effect_individuale_data.shape)
#protein_protein_data.head()

size of the protein-protein network: (499, 2)
size of the drug-target protein associations: (18690, 2)
size of the polypharmacy side effects: (9999, 4)


In [8]:
val_test_size = 0.1 #changed from 0.05 to 0.1 in the original code
#n_genes = 19081 #from protein-protein network 
#n_drugs = 645 #from drug-drug combo network

<h2> Creating a symmetric adjacency matrix for genes from the protein-protein network </h2>  

In [9]:
df_gene1_gene2 = pd.crosstab(protein_protein_data['Gene 1'], protein_protein_data['Gene 2'])
gene_idx = df_gene1_gene2.columns.union(df_gene1_gene2.index)
df_gene1_gene2 = df_gene1_gene2.reindex(index = gene_idx, columns = gene_idx,fill_value=0)#upper triangle of the adjacency matrix
df_gene2_gene1 = pd.crosstab(protein_protein_data['Gene 2'], protein_protein_data['Gene 1'])
df_gene2_gene1 = df_gene2_gene1.reindex(index = gene_idx, columns = gene_idx, fill_value=0)#lower triangle of the adjacency matrix
gene_adj = df_gene2_gene1.add(df_gene1_gene2, fill_value=0)#creates a symmetric adjacency matrix of Gene 1 and Gene 2 by adding upper triangle and lower triangle
gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()#get the degrees of genes
print("size of the gene-gene adjacecny matrix:",gene_adj.shape, ", number of genes:",len(gene_degrees))

size of the gene-gene adjacecny matrix: (496, 496) , number of genes: 496


In [10]:
n_genes = len(gene_degrees)

<h3>Making an adjacency matrix for the genes from the adjacency dataframe</h3>

In [11]:
indices_genes = list(range(0,len(gene_idx)))
dict_genes = dict(zip(gene_idx, indices_genes))#creating a dictionary to map genes to indices

In [12]:
gene_adj_mat = np.zeros((n_genes,n_genes)) # adj. matrix of size n_genes * n_genes
for i in range(0,protein_protein_data.shape[0]):#read from the protein-protein file
    gene1 = protein_protein_data.loc[i][0]#read gene1
    gene2 = protein_protein_data.loc[i][1]#read gene2
    #print(gene1,gene2)
    gene1_index = dict_genes.get(gene1)#get the index of gene1 in dictionary
    gene2_index = dict_genes.get(gene2)#get the index of gene2 in dictionary
    #print(gene1_index,gene2_index)
    gene_adj_mat[gene1_index][gene2_index] = 1.0 
    gene_adj_mat[gene2_index][gene1_index] = 1.0

In [13]:
G=nx.from_numpy_matrix(gene_adj_mat)
edges = G.edges(data=True)

In [14]:
def show_graph_with_labels(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500)
    plt.show()

In [15]:
#show_graph_with_labels(gene_adj_mat)

In [16]:
gene_adj_mat_t = gene_adj_mat.transpose() #transpose matrix of the adj. matrix 

<h2> Creating a list of symmetric adjacency matrices for each side effect from the drug-drug network (combo)</h2>

In [17]:
list_side_effects = []
prev_side_effect = []
for i in range(0,polypharmacy_side_effect_data.shape[0]):#get the list of side effects
    if polypharmacy_side_effect_data.loc[i][2] not in prev_side_effect:
        list_side_effects.append(polypharmacy_side_effect_data.loc[i][2])
        prev_side_effect.append(polypharmacy_side_effect_data.loc[i][2])
# ALTERNATIVELY BLA=set(polypharmacy_side_effect_data['Polypharmacy Side Effect'])

In [18]:
print("number of side effects =", len(list_side_effects) )
#polypharmacy_side_effect_data.head()



number of side effects = 995


In [19]:
#df_drug1_drug2 = pd.crosstab(polypharmacy_side_effect_data['STITCH 1'], polypharmacy_side_effect_data['STITCH 2'])
#drug_idx = df_drug1_drug2.columns.union(df_drug1_drug2.index)#get the list of all drugs
#n_drugs = len(drug_idx)
#print("number of drugs = ", len(drug_idx))

drug_set=set(polypharmacy_side_effect_data['STITCH 1']).union(set(polypharmacy_side_effect_data['STITCH 2']))
n_drugs=len(drug_set)
n_drugs



196

In [20]:
indices_drugs = list(range(0,n_drugs))
dict_drugs = dict(zip(drug_set, indices_drugs))#create a dictionary to map each drug to its index

<h2> Creating adjacency matrices for the drug-protein and protein-drug network.</h2>

In [27]:
#%%pixie_debugger
drug_drug_adj_list = []
for i in range(len(list_side_effects)):
    row = []
    col = []
    values = []
    drug_drug_mat = np.zeros((n_drugs,n_drugs))
    print("i=",i,"/",len(list_side_effects))
    for row_index in range(polypharmacy_side_effect_data.shape[0]):
        if polypharmacy_side_effect_data.loc[row_index][2] == list_side_effects[i]:
           # print(row_index)
            drug_1 = polypharmacy_side_effect_data.loc[row_index][0]
            drug_2 = polypharmacy_side_effect_data.loc[row_index][1] 
            drug_1_index = dict_drugs.get(drug_1)
            drug_2_index = dict_drugs.get(drug_2)
            #print(drug_1_index,drug_2_index)
            #drug_drug_mat[drug_1_index,drug_2_index] =  drug_drug_mat[drug_2_index,drug_1_index] = 1.
        #make sparse
            row.append(drug_1_index)
            col.append(drug_2_index)
            values.append(1.0)
            row.append(drug_2_index)
            col.append(drug_1_index)
            values.append(1.0)
            #print(row,col)
    mysparsematrix=sp.csr_matrix((values,(row,col)),shape=(n_drugs,n_drugs)).toarray()
    #print( mysparsematrix[drug_1_index,drug_2_index], mysparsematrix[drug_2_index,drug_1_index],drug_drug_mat[drug_1_index,drug_2_index],drug_drug_mat[drug_2_index,drug_1_index])
    #if (mysparsematrix==sp.csr_matrix(drug_drug_mat)).all():
          #print ("WOW")
    #else:
          #print ("TOO BAD")        
    drug_drug_adj_list.append(sp.csr_matrix(mysparsematrix))
    #drug_drug_adj_list.append(sp.csr_matrix(drug_drug_mat))
drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]

i= 0 / 995
[71, 60] [60, 71]
i= 1 / 995
[71, 60] [60, 71]
i= 2 / 995
[71, 60] [60, 71]
[71, 60, 116, 88] [60, 71, 88, 116]
[71, 60, 116, 88, 45, 187] [60, 71, 88, 116, 187, 45]
[71, 60, 116, 88, 45, 187, 140, 60] [60, 71, 88, 116, 187, 45, 60, 140]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99, 93, 53] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80, 53, 93]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99, 93, 53, 166, 43] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80, 53, 93, 43, 166]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99, 93, 53, 166, 43, 55, 158] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80, 53, 93, 43, 166, 158, 55]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99, 93, 53, 166, 43, 55, 158, 155, 138] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80, 53, 93, 43, 166, 158, 55, 138, 155]
[71, 60, 116, 88, 45, 187, 140, 60, 80, 99, 93, 53, 166, 43, 55, 158, 155, 138, 155, 51] [60, 71, 88, 116, 187, 45, 60, 140, 99, 80, 53, 

[71, 60, 116, 88, 168, 4, 45, 187, 55, 114, 77, 7, 87, 47, 166, 43, 192, 31, 55, 158, 120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 14, 124, 78, 182, 48, 58, 145, 95, 26, 153, 150, 176, 37, 145, 44, 152, 85, 49, 72, 40, 24, 129, 155, 36, 141, 65] [60, 71, 88, 116, 4, 168, 187, 45, 114, 55, 7, 77, 47, 87, 43, 166, 31, 192, 158, 55, 114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 124, 14, 182, 78, 58, 48, 95, 145, 153, 26, 176, 150, 145, 37, 152, 44, 49, 85, 40, 72, 129, 24, 36, 155, 65, 141]
[71, 60, 116, 88, 168, 4, 45, 187, 55, 114, 77, 7, 87, 47, 166, 43, 192, 31, 55, 158, 120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 14, 124, 78, 182, 48, 58, 145, 95, 26, 153, 150, 176, 37, 145, 44, 152, 85, 49, 72, 40, 24, 129, 155, 36, 141, 65, 131, 134] [60, 71, 88, 116, 4, 168, 187, 45, 114, 55, 7, 77, 47, 87, 43, 166, 31, 192, 158, 55, 114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 124, 14, 182, 78, 58, 48, 95, 145, 153, 26, 176, 150, 145, 37, 152, 44, 49, 85, 40, 72, 129, 24, 36, 155, 65, 141, 134, 1

[71, 60, 190, 75, 77, 7, 172, 182, 16, 3, 78, 182, 20, 55, 67, 174, 107, 137] [60, 71, 75, 190, 7, 77, 182, 172, 3, 16, 182, 78, 55, 20, 174, 67, 137, 107]
[71, 60, 190, 75, 77, 7, 172, 182, 16, 3, 78, 182, 20, 55, 67, 174, 107, 137, 80, 118] [60, 71, 75, 190, 7, 77, 182, 172, 3, 16, 182, 78, 55, 20, 174, 67, 137, 107, 118, 80]
[71, 60, 190, 75, 77, 7, 172, 182, 16, 3, 78, 182, 20, 55, 67, 174, 107, 137, 80, 118, 145, 95] [60, 71, 75, 190, 7, 77, 182, 172, 3, 16, 182, 78, 55, 20, 174, 67, 137, 107, 118, 80, 95, 145]
[71, 60, 190, 75, 77, 7, 172, 182, 16, 3, 78, 182, 20, 55, 67, 174, 107, 137, 80, 118, 145, 95, 89, 186] [60, 71, 75, 190, 7, 77, 182, 172, 3, 16, 182, 78, 55, 20, 174, 67, 137, 107, 118, 80, 95, 145, 186, 89]
[71, 60, 190, 75, 77, 7, 172, 182, 16, 3, 78, 182, 20, 55, 67, 174, 107, 137, 80, 118, 145, 95, 89, 186, 70, 108] [60, 71, 75, 190, 7, 77, 182, 172, 3, 16, 182, 78, 55, 20, 174, 67, 137, 107, 118, 80, 95, 145, 186, 89, 108, 70]
[71, 60, 190, 75, 77, 7, 172, 182, 16, 3

[71, 60, 132, 2, 116, 88] [60, 71, 2, 132, 88, 116]
[71, 60, 132, 2, 116, 88, 55, 114] [60, 71, 2, 132, 88, 116, 114, 55]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192, 138, 155]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138, 55, 157] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192, 138, 155, 157, 55]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138, 55, 157, 78, 182] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192, 138, 155, 157, 55, 182, 78]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138, 55, 157, 78, 182, 193, 162] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192, 138, 155, 157, 55, 182, 78, 162, 193]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138, 55, 157, 78, 182, 193, 162, 41, 164] [60, 71, 2, 132, 88, 116, 114, 55, 191, 192, 138, 155, 157, 55, 182, 78, 162, 193, 164, 41]
[71, 60, 132, 2, 116, 88, 55, 114, 192, 191, 155, 138,

[71, 60, 32, 136, 170, 0, 139, 143, 0, 163, 172, 182, 55, 157, 16, 3, 20, 55, 80, 118, 26, 153, 139, 19, 44, 152, 144, 28] [60, 71, 136, 32, 0, 170, 143, 139, 163, 0, 182, 172, 157, 55, 3, 16, 55, 20, 118, 80, 153, 26, 19, 139, 152, 44, 28, 144]
[71, 60, 32, 136, 170, 0, 139, 143, 0, 163, 172, 182, 55, 157, 16, 3, 20, 55, 80, 118, 26, 153, 139, 19, 44, 152, 144, 28, 24, 129] [60, 71, 136, 32, 0, 170, 143, 139, 163, 0, 182, 172, 157, 55, 3, 16, 55, 20, 118, 80, 153, 26, 19, 139, 152, 44, 28, 144, 129, 24]
[71, 60, 32, 136, 170, 0, 139, 143, 0, 163, 172, 182, 55, 157, 16, 3, 20, 55, 80, 118, 26, 153, 139, 19, 44, 152, 144, 28, 24, 129, 59, 176] [60, 71, 136, 32, 0, 170, 143, 139, 163, 0, 182, 172, 157, 55, 3, 16, 55, 20, 118, 80, 153, 26, 19, 139, 152, 44, 28, 144, 129, 24, 176, 59]
[71, 60, 32, 136, 170, 0, 139, 143, 0, 163, 172, 182, 55, 157, 16, 3, 20, 55, 80, 118, 26, 153, 139, 19, 44, 152, 144, 28, 24, 129, 59, 176, 146, 139] [60, 71, 136, 32, 0, 170, 143, 139, 163, 0, 182, 172, 157

...

[120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 67, 174, 48, 58, 145, 95, 156, 107, 64, 182, 24, 129] [114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 174, 67, 58, 48, 95, 145, 107, 156, 182, 64, 129, 24]
[120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 67, 174, 48, 58, 145, 95, 156, 107, 64, 182, 24, 129, 116, 171] [114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 174, 67, 58, 48, 95, 145, 107, 156, 182, 64, 129, 24, 171, 116]
[120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 67, 174, 48, 58, 145, 95, 156, 107, 64, 182, 24, 129, 116, 171, 149, 8] [114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 174, 67, 58, 48, 95, 145, 107, 156, 182, 64, 129, 24, 171, 116, 8, 149]
[120, 114, 155, 138, 55, 157, 126, 83, 16, 3, 67, 174, 48, 58, 145, 95, 156, 107, 64, 182, 24, 129, 116, 171, 149, 8, 106, 15] [114, 120, 138, 155, 157, 55, 83, 126, 3, 16, 174, 67, 58, 48, 95, 145, 107, 156, 182, 64, 129, 24, 171, 116, 8, 149, 15, 106]
i= 690 / 995
[120, 114] [114, 120]
[120, 114, 70, 108] [114, 120, 108, 70]
[120, 114, 70, 

[16, 3] [3, 16]
[16, 3, 107, 137] [3, 16, 137, 107]
[16, 3, 107, 137, 80, 118] [3, 16, 137, 107, 118, 80]
[16, 3, 107, 137, 80, 118, 92, 42] [3, 16, 137, 107, 118, 80, 42, 92]
[16, 3, 107, 137, 80, 118, 92, 42, 150, 176] [3, 16, 137, 107, 118, 80, 42, 92, 176, 150]
[16, 3, 107, 137, 80, 118, 92, 42, 150, 176, 138, 68] [3, 16, 137, 107, 118, 80, 42, 92, 176, 150, 68, 138]
[16, 3, 107, 137, 80, 118, 92, 42, 150, 176, 138, 68, 41, 34] [3, 16, 137, 107, 118, 80, 42, 92, 176, 150, 68, 138, 34, 41]
i= 729 / 995
[16, 3] [3, 16]
[16, 3, 150, 176] [3, 16, 176, 150]
[16, 3, 150, 176, 26, 21] [3, 16, 176, 150, 21, 26]
[16, 3, 150, 176, 26, 21, 78, 9] [3, 16, 176, 150, 21, 26, 9, 78]
[16, 3, 150, 176, 26, 21, 78, 9, 106, 15] [3, 16, 176, 150, 21, 26, 9, 78, 15, 106]
i= 730 / 995
[16, 3] [3, 16]
[16, 3, 78, 182] [3, 16, 182, 78]
[16, 3, 78, 182, 144, 73] [3, 16, 182, 78, 73, 144]
[16, 3, 78, 182, 144, 73, 107, 115] [3, 16, 182, 78, 73, 144, 115, 107]
[16, 3, 78, 182, 144, 73, 107, 115, 156, 107] [3

i= 771 / 995
[52, 163] [163, 52]
[52, 163, 107, 115] [163, 52, 115, 107]
[52, 163, 107, 115, 107, 137] [163, 52, 115, 107, 137, 107]
[52, 163, 107, 115, 107, 137, 29, 185] [163, 52, 115, 107, 137, 107, 185, 29]
[52, 163, 107, 115, 107, 137, 29, 185, 24, 129] [163, 52, 115, 107, 137, 107, 185, 29, 129, 24]
[52, 163, 107, 115, 107, 137, 29, 185, 24, 129, 41, 74] [163, 52, 115, 107, 137, 107, 185, 29, 129, 24, 74, 41]
[52, 163, 107, 115, 107, 137, 29, 185, 24, 129, 41, 74, 46, 118] [163, 52, 115, 107, 137, 107, 185, 29, 129, 24, 74, 41, 118, 46]
i= 772 / 995
[52, 163] [163, 52]
[52, 163, 145, 95] [163, 52, 95, 145]
i= 773 / 995
[52, 163] [163, 52]
i= 774 / 995
[52, 163] [163, 52]
[52, 163, 116, 171] [163, 52, 171, 116]
[52, 163, 116, 171, 41, 94] [163, 52, 171, 116, 94, 41]
[52, 163, 116, 171, 41, 94, 106, 15] [163, 52, 171, 116, 94, 41, 15, 106]
i= 775 / 995
[52, 163] [163, 52]
[52, 163, 160, 33] [163, 52, 33, 160]
[52, 163, 160, 33, 107, 115] [163, 52, 33, 160, 115, 107]
[52, 163, 160, 

[80, 118, 145, 95, 29, 185, 72, 40] [118, 80, 95, 145, 185, 29, 40, 72]
[80, 118, 145, 95, 29, 185, 72, 40, 155, 36] [118, 80, 95, 145, 185, 29, 40, 72, 36, 155]
[80, 118, 145, 95, 29, 185, 72, 40, 155, 36, 27, 169] [118, 80, 95, 145, 185, 29, 40, 72, 36, 155, 169, 27]
i= 825 / 995
[80, 118] [118, 80]
[80, 118, 150, 176] [118, 80, 176, 150]
[80, 118, 150, 176, 24, 129] [118, 80, 176, 150, 129, 24]
[80, 118, 150, 176, 24, 129, 59, 176] [118, 80, 176, 150, 129, 24, 176, 59]
i= 826 / 995
[80, 118] [118, 80]
[80, 118, 150, 176] [118, 80, 176, 150]
[80, 118, 150, 176, 139, 19] [118, 80, 176, 150, 19, 139]
[80, 118, 150, 176, 139, 19, 59, 176] [118, 80, 176, 150, 19, 139, 176, 59]
i= 827 / 995
[80, 118] [118, 80]
[80, 118, 150, 176] [118, 80, 176, 150]
[80, 118, 150, 176, 141, 190] [118, 80, 176, 150, 190, 141]
[80, 118, 150, 176, 141, 190, 59, 176] [118, 80, 176, 150, 190, 141, 176, 59]
i= 828 / 995
[80, 118] [118, 80]
[80, 118, 150, 176] [118, 80, 176, 150]
[80, 118, 150, 176, 59, 176] [11

[120, 195] [195, 120]
[120, 195, 24, 129] [195, 120, 129, 24]
i= 920 / 995
[120, 195] [195, 120]
[120, 195, 24, 129] [195, 120, 129, 24]
[120, 195, 24, 129, 91, 194] [195, 120, 129, 24, 194, 91]
i= 921 / 995
[120, 195] [195, 120]
i= 922 / 995
[64, 182] [182, 64]
i= 923 / 995
[144, 28] [28, 144]
[144, 28, 24, 129] [28, 144, 129, 24]
[144, 28, 24, 129, 106, 15] [28, 144, 129, 24, 15, 106]
i= 924 / 995
[144, 28] [28, 144]
[144, 28, 177, 182] [28, 144, 182, 177]
i= 925 / 995
[144, 28] [28, 144]
i= 926 / 995
[144, 28] [28, 144]
i= 927 / 995
[144, 28] [28, 144]
i= 928 / 995
[113, 23] [23, 113]
i= 929 / 995
[177, 182] [182, 177]
i= 930 / 995
[177, 182] [182, 177]
i= 931 / 995
[177, 182] [182, 177]
[177, 182, 135, 66] [182, 177, 66, 135]
i= 932 / 995
[24, 129] [129, 24]
i= 933 / 995
[24, 129] [129, 24]
i= 934 / 995
[24, 129] [129, 24]
i= 935 / 995
[24, 129] [129, 24]
[24, 129, 66, 57] [129, 24, 57, 66]
[24, 129, 66, 57, 146, 139] [129, 24, 57, 66, 139, 146]
i= 936 / 995
[24, 129] [129, 24]
i= 

In [28]:
drug_gene_adj = np.zeros((n_drugs,n_genes))
gene_drug_adj = np.zeros((n_genes, n_drugs))

In [29]:
#%%pixie_debugger
for i in range(0,drug_target_protein_data.shape[0]):
    drug = drug_target_protein_data.loc[i][0]
    gene = drug_target_protein_data.loc[i][1]
    if drug in drug_set and gene in gene_idx:
        gene_index = dict_genes.get(gene)
        drug_index = dict_drugs.get(drug)
        print(gene_index, drug_index)
        drug_gene_adj[drug_index][gene_index] = 1.0
        gene_drug_adj[gene_index][drug_index] = 1.0

321 181
139 181
134 181
461 181
495 181
222 181
72 181
92 181
255 181
48 181
241 181
482 181
240 181
118 181
225 181
288 181
64 181
107 181
254 181
197 181
386 181
391 181
136 181
476 181
200 181
10 181
493 181
41 181
400 181
492 181
95 181
396 181
102 181
50 181
100 181
16 181
170 181
184 181
276 181
494 181
453 181
484 181
351 181
7 181
67 181
4 181
123 181
122 181
12 181
55 181
56 181
98 181
49 181
51 181
261 181
248 181
207 181
125 181
124 181
1 181
479 181
324 181
294 181
376 181
253 181
192 181
133 90
0 45
351 45
44 45
116 146
44 102
178 149
127 175
227 175
0 175
319 147
0 147
265 147
127 147
227 147
351 147
349 147
44 147
133 3
324 153
116 58


In [30]:
drug_target_protein_data.shape[0]

18690

In [31]:
adj_mats_orig = {
    (0, 0): [sp.csr_matrix(gene_adj_mat), sp.csr_matrix(gene_adj_mat_t)],
    (0, 1): [sp.csr_matrix(gene_drug_adj)],
    (1, 0): [sp.csr_matrix(drug_gene_adj)],
    (1, 1): drug_drug_adj_list + [x.transpose(copy=True) for x in drug_drug_adj_list],
}

In [32]:
degrees = {
    0: [gene_degrees, gene_degrees],
    1: drug_degrees_list + drug_degrees_list,
}
# featureless (genes)
gene_feat = sp.identity(n_genes)
gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = preprocessing.sparse_to_tuple(gene_feat.tocoo())
# features (drugs)
drug_feat = sp.identity(n_drugs)
drug_nonzero_feat, drug_num_feat = drug_feat.shape
drug_feat = preprocessing.sparse_to_tuple(drug_feat.tocoo())
# data representation
num_feat = {
    0: gene_num_feat,
    1: drug_num_feat,
}
nonzero_feat = {
    0: gene_nonzero_feat,
    1: drug_nonzero_feat,
}
feat = {
    0: gene_feat,
    1: drug_feat,
}
edge_type2dim = {k: [adj.shape for adj in adjs] for k, adjs in adj_mats_orig.items()}
edge_type2decoder = {
    (0, 0): 'bilinear',
    (0, 1): 'bilinear',
    (1, 0): 'bilinear',
    (1, 1): 'dedicom',
}

In [33]:
edge_types = {k: len(v) for k, v in adj_mats_orig.items()}
num_edge_types = sum(edge_types.values())
print("Edge types:", "%d" % num_edge_types)

###########################################################
#
# Settings and placeholders
#
###########################################################

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer('neg_sample_size', 1, 'Negative sample size.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 50, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0.1, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('max_margin', 0.1, 'Max margin parameter in hinge loss')
flags.DEFINE_integer('batch_size', 512, 'minibatch size.')
flags.DEFINE_boolean('bias', True, 'Bias term.')
# Important -- Do not evaluate/print validation performance every iteration as it can take
# substantial amount of time
PRINT_PROGRESS_EVERY = 150

print("Defining placeholders")
placeholders = construct_placeholders(edge_types)

Edge types: 1994
Defining placeholders


In [34]:
print("Create minibatch iterator")
minibatch = EdgeMinibatchIterator(
    adj_mats=adj_mats_orig,
    feat=feat,
    edge_types=edge_types,
    batch_size=FLAGS.batch_size,
    val_test_size=val_test_size
)

Create minibatch iterator
Minibatch edge type: (0, 0, 0)
Constructing test edges= 0000/0099
Constructing val edges= 0000/0099
Train edges= 0800
Val edges= 0099
Test edges= 0099
Minibatch edge type: (0, 0, 1)
Constructing test edges= 0000/0099
Constructing val edges= 0000/0099
Train edges= 0800
Val edges= 0099
Test edges= 0099
Minibatch edge type: (0, 1, 0)
Constructing test edges= 0000/0037
Constructing val edges= 0000/0050
Train edges= 0000
Val edges= 0050
Test edges= 0037
Minibatch edge type: (1, 0, 0)
Constructing test edges= 0000/0037
Constructing val edges= 0000/0050
Train edges= 0000
Val edges= 0050
Test edges= 0037
Minibatch edge type: (1, 1, 0)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 2)
Constructing val edges= 0000/0048
Train edges= 0000
Val edges= 0048
Test edges= 0000
Minibatch edge type: (

/Users/ravanv/Desktop/Decagon/decagon-master/decagon/deep/minibatch.py:67: RuntimeWarning: divide by zero encountered in power
  rowdegree_mat_inv = sp.diags(np.nan_to_num(np.power(rowsum, -0.5)).flatten())
/Users/ravanv/Desktop/Decagon/decagon-master/decagon/deep/minibatch.py:68: RuntimeWarning: divide by zero encountered in power
  coldegree_mat_inv = sp.diags(np.nan_to_num(np.power(colsum, -0.5)).flatten())


Test edges= 0030
Minibatch edge type: (1, 1, 5)
Constructing val edges= 0000/0042
Train edges= 0000
Val edges= 0042
Test edges= 0000
Minibatch edge type: (1, 1, 6)
Constructing test edges= 0000/0008
Constructing val edges= 0000/0050
Train edges= 0000
Val edges= 0050
Test edges= 0008
Minibatch edge type: (1, 1, 7)
Constructing val edges= 0000/0038
Train edges= 0000
Val edges= 0038
Test edges= 0000
Minibatch edge type: (1, 1, 8)
Constructing val edges= 0000/0028
Train edges= 0000
Val edges= 0028
Test edges= 0000
Minibatch edge type: (1, 1, 9)
Constructing val edges= 0000/0020
Train edges= 0000
Val edges= 0020
Test edges= 0000
Minibatch edge type: (1, 1, 10)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 11)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 12)
Constructing val edges= 0000/0042
Train edges= 0000
Val edges= 0042
Test edges= 0000
Minibatch edge ty

Val edges= 0034
Test edges= 0000
Minibatch edge type: (1, 1, 159)
Constructing val edges= 0000/0018
Train edges= 0000
Val edges= 0018
Test edges= 0000
Minibatch edge type: (1, 1, 160)
Constructing test edges= 0000/0006
Constructing val edges= 0000/0050
Train edges= 0000
Val edges= 0050
Test edges= 0006
Minibatch edge type: (1, 1, 161)
Constructing val edges= 0000/0016
Train edges= 0000
Val edges= 0016
Test edges= 0000
Minibatch edge type: (1, 1, 162)
Constructing val edges= 0000/0038
Train edges= 0000
Val edges= 0038
Test edges= 0000
Minibatch edge type: (1, 1, 163)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 164)
Constructing val edges= 0000/0036
Train edges= 0000
Val edges= 0036
Test edges= 0000
Minibatch edge type: (1, 1, 165)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 166)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test 

Minibatch edge type: (1, 1, 253)
Constructing val edges= 0000/0020
Train edges= 0000
Val edges= 0020
Test edges= 0000
Minibatch edge type: (1, 1, 254)
Constructing val edges= 0000/0020
Train edges= 0000
Val edges= 0020
Test edges= 0000
Minibatch edge type: (1, 1, 255)
Constructing val edges= 0000/0014
Train edges= 0000
Val edges= 0014
Test edges= 0000
Minibatch edge type: (1, 1, 256)
Constructing val edges= 0000/0004
Train edges= 0000
Val edges= 0004
Test edges= 0000
Minibatch edge type: (1, 1, 257)
Constructing val edges= 0000/0016
Train edges= 0000
Val edges= 0016
Test edges= 0000
Minibatch edge type: (1, 1, 258)
Constructing val edges= 0000/0050
Train edges= 0000
Val edges= 0050
Test edges= 0000
Minibatch edge type: (1, 1, 259)
Constructing val edges= 0000/0040
Train edges= 0000
Val edges= 0040
Test edges= 0000
Minibatch edge type: (1, 1, 260)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 261)
Constructing val edges=

...

Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 1542)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1543)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 1544)
Constructing val edges= 0000/0026
Train edges= 0000
Val edges= 0026
Test edges= 0000
Minibatch edge type: (1, 1, 1545)
Constructing val edges= 0000/0022
Train edges= 0000
Val edges= 0022
Test edges= 0000
Minibatch edge type: (1, 1, 1546)
Constructing val edges= 0000/0026
Train edges= 0000
Val edges= 0026
Test edges= 0000
Minibatch edge type: (1, 1, 1547)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1548)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1549)
Constructing val edges= 0000/0032
Train edges= 0

Train edges= 0000
Val edges= 0012
Test edges= 0000
Minibatch edge type: (1, 1, 1652)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1653)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 1654)
Constructing val edges= 0000/0018
Train edges= 0000
Val edges= 0018
Test edges= 0000
Minibatch edge type: (1, 1, 1655)
Constructing val edges= 0000/0012
Train edges= 0000
Val edges= 0012
Test edges= 0000
Minibatch edge type: (1, 1, 1656)
Constructing val edges= 0000/0014
Train edges= 0000
Val edges= 0014
Test edges= 0000
Minibatch edge type: (1, 1, 1657)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1658)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1659)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 00

Test edges= 0000
Minibatch edge type: (1, 1, 1754)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 1755)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1756)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 1757)
Constructing val edges= 0000/0008
Train edges= 0000
Val edges= 0008
Test edges= 0000
Minibatch edge type: (1, 1, 1758)
Constructing val edges= 0000/0004
Train edges= 0000
Val edges= 0004
Test edges= 0000
Minibatch edge type: (1, 1, 1759)
Constructing val edges= 0000/0010
Train edges= 0000
Val edges= 0010
Test edges= 0000
Minibatch edge type: (1, 1, 1760)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1761)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 176

Train edges= 0000
Val edges= 0004
Test edges= 0000
Minibatch edge type: (1, 1, 1857)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1858)
Constructing val edges= 0000/0004
Train edges= 0000
Val edges= 0004
Test edges= 0000
Minibatch edge type: (1, 1, 1859)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1860)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 0000
Minibatch edge type: (1, 1, 1861)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1862)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1863)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1864)
Constructing val edges= 0000/0006
Train edges= 0000
Val edges= 0006
Test edges= 00

Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1977)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1978)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1979)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1980)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1981)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1982)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1983)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 0000
Minibatch edge type: (1, 1, 1984)
Constructing val edges= 0000/0002
Train edges= 0000
Val edges= 0002
Test edges= 00

In [35]:
print("Create model")
model = DecagonModel(
    placeholders=placeholders,
    num_feat=num_feat,
    nonzero_feat=nonzero_feat,
    edge_types=edge_types,
    decoders=edge_type2decoder,
)

Create model


In [36]:
print("Create optimizer")
with tf.name_scope('optimizer'):
    opt = DecagonOptimizer(
        embeddings=model.embeddings,
        latent_inters=model.latent_inters,
        latent_varies=model.latent_varies,
        degrees=degrees,
        edge_types=edge_types,
        edge_type2dim=edge_type2dim,
        placeholders=placeholders,
        batch_size=FLAGS.batch_size,
        margin=FLAGS.max_margin
    )

Create optimizer


/Users/ravanv/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
print("Initialize session")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {}

Initialize session


In [ ]:
###########################################################
#
# Train model
#
###########################################################

print("Train model")
for epoch in range(FLAGS.epochs):

    minibatch.shuffle()
    itr = 0
    while not minibatch.end():
        print("iteration:", itr)
        # Construct feed dictionary
        feed_dict = minibatch.next_minibatch_feed_dict(placeholders=placeholders)
        feed_dict = minibatch.update_feed_dict(
            feed_dict=feed_dict,
            dropout=FLAGS.dropout,
            placeholders=placeholders)

        t = time.time()

        # Training step: run single weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.batch_edge_type_idx], feed_dict=feed_dict)
        train_cost = outs[1]
        batch_edge_type = outs[2]

        if itr % PRINT_PROGRESS_EVERY == 0:
            val_auc, val_auprc, val_apk = get_accuracy_scores(
                minibatch.val_edges, minibatch.val_edges_false,
                minibatch.idx2edge_type[minibatch.current_edge_type_idx])

            print("Epoch:", "%04d" % (epoch + 1), "Iter:", "%04d" % (itr + 1), "Edge:", "%04d" % batch_edge_type,
                  "train_loss=", "{:.5f}".format(train_cost),
                  "val_roc=", "{:.5f}".format(val_auc), "val_auprc=", "{:.5f}".format(val_auprc),
                  "val_apk=", "{:.5f}".format(val_apk), "time=", "{:.5f}".format(time.time() - t))

        itr += 1

print("Optimization finished!")

Train model
iteration: 0
Epoch: 0001 Iter: 0001 Edge: 0000 train_loss= 73.70885 val_roc= 0.44230 val_auprc= 0.46543 val_apk= 0.20317 time= 64.24864
iteration: 1


In [ ]:
for et in range(num_edge_types):
    roc_score, auprc_score, apk_score = get_accuracy_scores(
        minibatch.test_edges, minibatch.test_edges_false, minibatch.idx2edge_type[et])
    print("Edge type=", "[%02d, %02d, %02d]" % minibatch.idx2edge_type[et])
    print("Edge type:", "%04d" % et, "Test AUROC score", "{:.5f}".format(roc_score))
    print("Edge type:", "%04d" % et, "Test AUPRC score", "{:.5f}".format(auprc_score))
    print("Edge type:", "%04d" % et, "Test AP@k score", "{:.5f}".format(apk_score))
    print()
